## GAN

summary：**a minmax game theory leads to minimize Jensen-Shannon divergence**

####  1. generator $p_\theta$

定义在support space $\mathcal{X}$ 上的两个分布 $p_r(x)$， $p_\theta(x)$。不妨把 $\forall x \in \mathcal{X}$ 想象成描述图片像素值分布的多维向量，这两个分布分别对应“真实”图片的分布，和机器生成的“仿真”图片的分布。“真实”图片的集合，我们用 $y=1$ 标记；“仿真”图片的集合，我们用 $y=0$ 标记，则我们总可以定义如下关于所有图片（包括“真实的”和“仿真的”）的联合概率密度：$\mathbb{p}_\theta(x,y) = p_{y=0} p_\theta(x)\delta_{y,0} + p_{y=1} p_r(x)\delta_{y,1}$, 在没有任何类别bias的前提下，我们不妨取 $p_{y=0}=p_{y=1}={1/2}$, 于是有：

$$\mathbb{p}_\theta(x,y) = {1\over2} p_\theta(x)\delta_{y,0} + {1\over2} p_r(x)\delta_{y,1}$$

>为了便捷的生成“仿真”sample，$p_\theta$ 将按如下方式构造：给定一个已知其分布的随机变量$z\sim \zeta(z)$ (通常可取为标准正态分布)，再用一个函数 $x=G_\theta(z)$ 来生成 $x$。函数 $G_\theta(\cdot)$ 用一个前馈神经网络来表征,其参数用 $\theta$ 标记。

我们将 $\mathbb{p}_\theta(x,y)$ 看做一个生成 $(x, y)$ pair 的机器 （当 $y=1$, 生成的图片为“真实”图片，当$y=0$ 生成的图片为“仿真”图片）


#### 2. discriminator $q_\phi$

给定一个$x$, 我们可以用一个前馈神经网络（参数记为$\phi$）来输出该图片属于“真实”类（$y=0$）还是“仿真类”（$y=1$）的概率，即 $(x, y)$ 组成“pair”的概率。该神经网络的作用，在数学上与条件概率的意义相当，因此记为 $q_\phi(y|x)$


####  3. balance between generator and discriminator


对于给定的生成机（即$\theta$固定），辨别机的目的是：尽可能分辨“真伪”，即符合生成机给出的 pair 关系，即：

Given generator, the discriminator tunes its parameter $\phi$  to maximize value function
$V(p_\theta, q_\phi)\equiv\mathbb{E}_{\mathbb{p}_\theta(x,y)}\left[\log q_\phi(y|x)\right]$

对于给定的辨别机（即$\phi$固定），生成机的目的是：尽可能让辨别机失效，即：

Given discriminator, the generator tunes its parameter $\theta$ to minimize value function 
$V(p_\theta, q_\phi)$

二者达到“纳什均衡”后，博弈停止。

#### 4. 博弈的“纳什均衡”解，和 minimize Jensen-Shannon divergence 的解一致

$$\begin{align}
V(p_\theta, q_\phi)\equiv\mathbb{E}_{\mathbb{p}_\theta(x,y)}\left[\log q_\phi(y|x)\right] &={1\over2}\left(\int{\rm d}x p_r(x) q_\phi(y=1|x) + \int {\rm d}x p_\theta(x) q_\phi(y=0|x)\right)\\
&={1\over2}\int {\rm d}x\left[p_r(x)\log D_\phi(x) + p_\theta(x)\log\left(1- D_\phi(x)\right)\right]
,\qquad\text{其中 $D_\phi(x)\equiv q_\phi(y=1|x) $}\end{align}$$

给定 $p_\theta(x)$ 的函数形式（即给定生成机），可以证明当 $D_\phi(x)={p_r(x)\over p_r(x) + p_\theta(x)}$，可以 maximize $V$， 即 生成机 $ p_\theta$ 和 辨别机 $q_\phi$ 达到纳什均衡的一个必要条件是：
$$D_\phi(x)={p_r(x)\over p_r(x) + p_\theta(x)}$$

将此关系，代入value function，我们有：

$$\begin{align}
V
&={1\over2}\int {\rm d}x\left[p_r(x)\log {p_r(x)\over p_r(x) + p_\theta(x)}+ p_\theta(x)\log {p_\theta(x)\over p_r(x) + p_\theta(x)}\right]\\
&={1\over2}\int {\rm d}x\left[p_r(x)\log {2p_r(x)\over p_r(x) + p_\theta(x)}+ p_\theta(x)\log {2p_\theta(x)\over p_r(x) + p_\theta(x)}-\log4\right]\\
&={1\over2}\left[\int {\rm d}xp_r(x)\left(\log p_r(x) - \log{ p_r(x) + p_\theta(x)\over2}\right)+
\int {\rm d}xp_\theta(x)\left(\log p_\theta(x) - \log{ p_r(x) + p_\theta(x)\over2}\right)\right] -\log2\\
&={1\over2}\left[KL\left(p_r(x)\left\|{ p_r(x) + p_\theta(x)\over2}\right.\right)+
KL\left(p_\theta(x)\left\|{ p_r(x) + p_\theta(x)\over2}\right.\right)\right] - \log2\\
&=JSD\left(\left.p_r\right\|p_\theta\right) - \log2,\qquad\text{其中JSD 称为 Jensen-Shannon divergence}\end{align}$$

生成机 $ p_\theta$ 和 辨别机 $q_\phi$ 达到纳什均衡的一个必要条件是, $p_\theta$ 要 minimize $V$

因此博弈的“纳什均衡”解，等价于 minimize Jensen-Shannon divergence 的解

## WGAN

summary：**minimize Wasserstein divergence leads to a minmax game theory**


####  1. Wasserstein divergence 的定义
定义在support space $\mathcal{X}$ 上的两个分布 $\mathbb{p}_r$， $\mathbb{p}_\theta$ （注意：这两个分布未必可以用smooth的density来描述。含有Dirac function的概率密度不是smooth的，因而无法用SGD等方法找到最优解），记二者的divergence (distance) 为 $\rho(\mathbb{p}_r,\mathbb{p}_\theta)$ 。 $\rho(\cdot)$ 应当满足如下“合理”条件 

1. 正定，且 仅当$\mathbb{p}_r=\mathbb{p}_\theta$ 时， $\rho=0$
2. $\mathbb{p}_r$, $\mathbb{p}_\theta$ 差异越大，$\rho(\mathbb{p}_r,\mathbb{p}_\theta)$ 越大

满足这个条件，可以定义多种形式的"divergence"。

* Kullback-Leibler 形式： $$\rho(\mathbb{p}_r,\mathbb{p}_\theta) \equiv\left\{\begin{align} KL(\mathbb{p}_r\|\mathbb{p}_\theta)&\equiv& \sum\limits_{x\in\mathcal{X}} p_r(x)\left(\log p_r(x) - \log p_\theta(x)\right)\\
&\text{或}&\\
KL(\mathbb{p}_\theta\|\mathbb{p}_r)&\equiv& \sum\limits_{x\in\mathcal{X}} p_\theta(x)\left(\log p_\theta(x) - \log p_r(x)\right)\end{align}\right.$$

* Jensen-Shannon 形式：
    
$$\rho(\mathbb{p}_r,\mathbb{p}_\theta)\equiv JS(\mathbb{p}_r,\mathbb{p}_\theta) = KL(\mathbb{p}_r\|\mathbb{p}_m) + KL(\mathbb{p}_\theta\|\mathbb{p}_m),\quad\text{其中 }\mathbb{p}_m = {\mathbb{p}_r+\mathbb{p}_\theta\over2}$$

* Total Varation 形式：

$$\rho(\mathbb{p}_r,\mathbb{p}_\theta) \equiv \delta(\mathbb{p}_r,\mathbb{p}_\theta)= \max\limits_{A\subseteq \mathcal{X}}\left\|\mathbb{p}_r(A) - \mathbb{p}_\theta(A) \right\|,\quad\text{其中 $A$ 是 support space $\mathcal{X}$ 的子集，$\mathbb{p}(A)$ 表示 $A$ 的概率值, max 表述取最大值}$$

* Wasserstein 形式：

$$\rho(\mathbb{p}_r,\mathbb{p}_\theta) \equiv W(\mathbb{p}_r,\mathbb{p}_\theta) = \min\limits_{\gamma\in\Pi(\mathbb{p}_r,\mathbb{p}_\theta)}\;\mathbb{E}_{(x,y)\sim\gamma}\;\left\|x-y \right\|,\quad\text{其中 $\Pi(\mathbb{p}_r,\mathbb{p}_\theta)$ 表示所有“marginal分布分别为 $\mathbb{p}_r$ 和 $\mathbb{p}_\theta$” 的joint分布集合 }$$

#### 2. Wasserstein divergence 的良好性质

举例如下：设 $\mathbb{p}_0$ 和 $\mathbb{p}_\theta$ 是定义在如下grid 上的两个分布，grid上的每个点用 $(x_0,x_1)$ 标记：$\mathbb{p}_0$ 在 $x_0=0$ 上的点取概率值 $1/N$,其他概率为0；$\mathbb{p}_\theta$ 在 $x_0=\theta$ 上的点取概率值 $1/N$,其他概率为0 ($\theta\ne 0$, 即 $\mathbb{p}_0$ 与 $\mathbb{p}_\theta$ 不重合).

![p_0](./images/wasserstein0.jpg)
![p_theta](./images/wasserstein1.jpg)

将$x_0=0$ 和 $x_0=\theta$ 对应的点集分别记为 $L_0$、$L_\theta$， 则  $\mathbb{p}_0$ 和 $\mathbb{p}_\theta$ 的各种 divergence 值分别为：

$$\begin{align}
KL(\mathbb{p}_0\|\mathbb{p}_\theta) &=& \sum\limits_{x\in L_0} {1\over N} \left(\log {1\over N}- \log 0\right) = +\infty\\
KL(\mathbb{p}_\theta\|\mathbb{p}_0) &=& \sum\limits_{x\in L_\theta} {1\over N} \left(\log {1\over N}- \log 0\right) = +\infty
\end{align}$$

$$\begin{align}
JS(\mathbb{p}_0,\mathbb{p}_\theta) &= &{ \sum\limits_{x\in L_0} {1\over N} \left(\log {1\over N}- \log {1\over 2N}\right) + \sum\limits_{x\in L_\theta} {1\over N} \left(\log {1\over N}- \log {1\over 2N}\right)\over 2} = {(\log 2N -\log N) + (\log 2N -\log N)\over 2} =\log2
\end{align}$$

$$\begin{align}
\delta(\mathbb{p}_r,\mathbb{p}_\theta)&=& \max\limits_{A\subseteq \mathcal{X}}\left\|\mathbb{p}_r(A) - \mathbb{p}_\theta(A) \right\| = \left(\left\|\mathbb{p}_r(A) - \mathbb{p}_\theta(A) \right\|\right)_{A=L_0}\; = \|1-0\|=1
\end{align}$$

$$ W(\mathbb{p}_0,\mathbb{p}_\theta) = |\theta|$$

> 显然，可以构造两个属于 $\Pi(\mathbb{p}_0, \mathbb{p}_\theta)$ 的分布：

> $$\gamma_0(x,y)\equiv \gamma_0(x_0,x_1, y_0,y_1) = \left\{\begin{align}
0,&\quad \text{当 $x_0\ne 0$ 或 $y_0\ne\theta$} \\
{1\over N^2},& \quad\text{其他}
\end{align}\right.$$

> $$\gamma_1(x,y)\equiv \gamma_0(x_0,x_1, y_0,y_1) = \left\{\begin{align}
0,&\quad \text{当 $x_0\ne 0$ 或 $y_0\ne\theta$} \\
{1\over N}\delta_{x_1,y_1},& \quad\text{其他}
\end{align}\right.$$

> （显然对于 $\forall \alpha,\beta\ge0,\alpha + \beta=1$， $\alpha\gamma_0(x,y) + \beta\gamma_1(x,y)$ 也属于集合 $\Pi(\mathbb{p}_0, \mathbb{p}_\theta)$） 

> $$\mathbb{E}_{(x,y)\sim\gamma_0}\;\left\|x-y \right\| =\mathbb{E}_{(x,y)\sim\gamma_0}\;\left\|(0,x_1)-(\theta,y_1)\right\|  = {1\over N^2}\sum_{i=1}^N\sum_{j=1}^N\sqrt{\theta^2 + \left({i-j\over N}\right)^2}>|\theta|$$

> $$\mathbb{E}_{(x,y)\sim\gamma_1}\;\left\|x-y \right\| =\mathbb{E}_{(x,y)\sim\gamma_1}\;\left\|(0,x_1)-(\theta,y_1)\right\| ={1\over N} \sum_{x_1,y_1} \delta_{x_1,y_1}|\theta| = {1\over N} \sum_{x_1} |\theta| =|\theta| $$
 
> 事实上, $\gamma_1$ 恰是使 $\mathbb{E}_{(x,y)\sim\gamma_0}\;\left\|x-y \right\| $ 取最小值的分布，即 $ W(\mathbb{p}_0,\mathbb{p}_\theta) = |\theta|$


通过上面的例子，我们发现 KL divergence 的“连续性”最差（即，若希望它保持良好解析性质，对 $\mathbb{p}_0$ 与 $\mathbb{p}_\theta$ 之间的关系要求较高） ；JS divergence 和 Total Varation 的“连续性”次之； ** Wasserstein divergence“连续性”最好。“连续性”好意味着，更容易用SGD方法训练相关divergence **

> 事实上，正是由于KL divergence 对于在低维manifold上的两个分布，很可能出现 infinity值， 因此基于KL divergence的训练，往往需要引入遍布全空间的random noise才可以良好的进行

#### 3. minimize Wasserstein divergence 化为 minmax game形式

将 $\mathbb{p}_r $ 和 $\mathbb{p}_\theta$ 分别理解成data的“真实”分布 和模型的理论分布（$\theta$ 为模型参数），则我们训练的目标是：

$$\theta^* = \mathop{\rm argmin}\limits_{\theta} \rho(\mathbb{p}_r, \mathbb{p}_\theta)$$



我们取 $$\rho(\mathbb{p}_r, \mathbb{p}_\theta)\equiv  K\cdot W(\mathbb{p}_r,\mathbb{p}_\theta) $$，其中 $K>0$ 是一个取定的常数，则可以证明：

$$\rho(\mathbb{p}_r, \mathbb{p}_\theta) = \max\limits_{f\in L_K}\left\{\mathbb{E}_{x\sim \mathbb{p}_r}\;[f(x)] - \mathbb{E}_{x\sim \mathbb{p}_\theta}\;[f(x)]\right\}$$

其中 $L_K$ 表示 所有满足 $K$-Lipschitz 连续性条件的函数集合

为涵盖 $x\sim\mathbb{p}_\theta$ 分布为一个低维 manifold 的可能性, 我们**不假定 ** $\mathbb{p}_\theta$ 可以用一个全空间 smooth 的density来描述， 而是假定 $x\sim \mathbb{p}_\theta$ 是按如下方式构造的：

$$z\sim p(z),\qquad x=g_\theta(z)$$

其中 $p(z)$ 是一个给定的**smooth** density函数，$g_\theta(\cdot)$ 是将随机变量$z$ 映射为$x$的函数（用一个参数为$\theta$的前馈神经网络表征，称为generator）

$$\rho(\mathbb{p}_r, \mathbb{p}_\theta) = \max\limits_{f\in L_K}\left\{\mathbb{E}_{x\sim \mathbb{p}_r}\;[f(x)] - \mathbb{E}_{z\sim p(z)}\;[f(g_\theta(z))]\right\}$$

用一个参数记为 $\omega$ 的前馈神经网络（称为critic/discriminator）来“拟合” $f(\cdot)$，即 $f(x)\rightarrow f_\omega(x)$, $\omega$ 为模型参数。即，
 $$\begin{align}\omega^* &= \mathop{\rm argmax}\limits_{\omega:\; f_\omega\in L_K} \;\left\{\mathbb{E}_{x\sim \mathbb{p}_r}\;[f_\omega(x)] - \mathbb{E}_{z\sim p(z)}\;[f_\omega(g_\theta(z))]\right\}\\
\rho(\mathbb{p}_r, \mathbb{p}_\theta) &= \mathbb{E}_{x\sim \mathbb{p}_r}\;[f(x,\omega^*)] - \mathbb{E}_{z\sim p(z)}\;[f(g(z,\theta),\omega^*)]\\
\theta^* &= \mathop{\rm argmin}\limits_{\theta} \rho(\mathbb{p}_r, \mathbb{p}_\theta)
\end{align}$$

或者简记为：

$$(\theta^*,\omega^*) = \mathop{\rm argmin}\limits_{\theta} \mathop{\rm argmax}\limits_{\omega:\; f_\omega\in L_K} \;\left\{\mathbb{E}_{x\sim \mathbb{p}_r}\;[f_\omega(x)] - \mathbb{E}_{z\sim p(z)}\;[f_\omega(g_\theta(z))]\right\}$$


cGAN: conditional adversarial networks